# COMP541 - LAB #5

<img src="http://drive.google.com/uc?export=view&id=1RJUAUMmViCsYSgyFj7oyqBu4c6AZMeFR">

T-Maze problem was designed to test Reinforcement Learning-LSTM's capability to bridge long time lags, without confounding the results by making the control task difficult in other ways [(Bakker, 2002)](http://papers.nips.cc/paper/1953-reinforcement-learning-with-long-short-term-memory.pdf]). However, we can also use this test-bed for supervised algorithms by exploiting the gold actions.

In the T-Maze problem, the agent has four possible actions: move North, East, South, or West. The agent must learn to move from the starting position at the beginning of the corridor to the T-junction. There it must move either North or South to a changing goal position, which it cannot see. However, the location of the goal depends on a "road sign" the agent has seen at the starting position. At the starting position, the observation is either 011 (North) or 110 (South), in the corridor the observation is 101, and at the T-junction the observation is 010.

In this assignment, you will complete lstm based architecture, train and test the model on different settings to inspect the learning curve and generalization power of the model. You will also explore the behavior of the model by visualizing hidden and cell vectors, and weights.

In [ ]:
using Pkg;

# Install missing packages
for p in ["Knet", "Luxor", "Plots"]
    if !haskey(Pkg.installed(),p)
        Pkg.add(p);
    end
end

using Knet, Luxor, Plots, Random, Printf

## General Information
Our goal is 0 for North and 1 for South
Agent can take an action with step function. Directions are 1 for North, 2 for East, 3 for South, and 4 for West. If the action is not permitted, nothing happens in that round. These actions are:
* Taking action North, South or West at the starting point
* Taking action South or North in the corridor
* Taking action East at the T-junction

At each step, our input is as follows:
* At the starting point
  + If end goal is South -> \[1.0, 1.0, 0.0\]
  + If end goal is North -> \[0.0, 1.0, 1.0\]
* In the corridor -> \[1.0, 0.0, 1.0\]
* At the T-junction -> \[0.0, 1.0, 0.0\]

In [ ]:
mutable struct TMaze
    length::Int # length except T-junction
    goal::Int # 0 = North, 1 = South
    agent_position::Tuple{Int,Int} # Agent's current position (x, y)
end

function get_state(maze::TMaze)
    """
    Returns the current state
    """

    state = zeros(Float32, 3,)

    if maze.agent_position[1] == maze.length # At the T-junction
        state[2] = 1.0
    elseif maze.agent_position[1] == 0 # At the start position
        if maze.goal == 0
            state[2] = 1.0
            state[3] = 1.0
        else
            state[1] = 1.0
            state[2] = 1.0
        end
    else # In the corridor
        state[1] = 1.0
        state[3] = 1.0
    end
    return state
end

function step(maze::TMaze, action::Int)
    """
    Gets action and plays one step. If the resulted state is the final state
    then it checks whether it is the goal state or not. If it is a goal state
    and the correct one it returns 1, if it is wrong it returns -1.
    If the resulted state is not the final state, then it returns 0.
    1 : North
    2 : East
    3 : South
    4 : West
    """
    p = maze.agent_position
    res = 0

    if p[1] == maze.length && p[2] == 0
        if action == 1
            maze.agent_position = (p[1], min(-1, p[2]-1))
            res = maze.goal == 0 ? 1 : -1
        elseif action == 3
            maze.agent_position = (p[1], min(1, p[2]+1))
            res = maze.goal == 0 ? -1 : 1
        elseif action == 4
            maze.agent_position = (p[1]-1, 0)
        end
    elseif p[1] != maze.length && action == 2
        maze.agent_position = (p[1]+1, 0)
    elseif p[1] != maze.length && action == 4
        maze.agent_position = (min(p[1]-1, 0), 0)
    end
    return res
end

function get_gold_actions(maze::TMaze)
    """
    Returns a vector of integer for gold actions
    """

    gold_actions = ones(Int, maze.length) * 2
    push!(gold_actions, maze.goal == 0 ? 1 : 3)
    return gold_actions
end

function get_supervised_states(maze::TMaze)
    """
    Returns the list of states for the gold actions.
    The list includes the starting position, but does not include the final state.
    """

    p = maze.agent_position # store the agent position

    maze.agent_position = (0, 0)

    states = []
    for action in get_gold_actions(maze)
        push!(states, get_state(maze))
        step(maze, action)
    end

    maze.agent_position = p # reset
    return states
end

function draw(maze::TMaze)
    dim = 500 / (2 * (maze.length +1))
    Drawing(500, round(Int, dim*3))
    origin()
    background("white")
    sethue("black")
    box.([O + (i*dim, 0) for i=0:maze.length], dim, dim, :stroke)
    if maze.goal == 0
        sethue("crimson")
        box.(O + (maze.length*dim, -dim), dim, dim, :fill)
        sethue("black")
        box.(O + (maze.length*dim, dim), dim, dim, :stroke)
        if maze.agent_position[1] == 0
            Luxor.arrow(O + (0, -dim*0.65), Point(0, -dim*1.45))
        end
    else
        sethue("crimson")
        box.(O + (maze.length*dim, dim), dim, dim, :fill)
        sethue("black")
        box.(O + (maze.length*dim, -dim), dim, dim, :stroke)
        if maze.agent_position[1] == 0
            Luxor.arrow(O + (0, dim*0.65), Point(0, dim*1.45))
        end
    end
    sethue("steelblue4")
    circle(O + (maze.agent_position[1]*dim,maze.agent_position[2]*dim), dim*0.45, :fill)
    finish()
    display(preview())
    IJulia.clear_output(true)
end

In [ ]:
maze = TMaze(5,1,(0,0));

In [ ]:
draw(maze);

In [ ]:
maze2 = TMaze(5,1,(0,0));
draw(maze2)
sleep(1)

# Let's animate
for action in get_gold_actions(maze2)
    res = step(maze2, action)
    draw(maze2)
    # print("Action: $action\n")
    # print("Result: $res")
    sleep(1) # sleep 1 second
end

In [ ]:
get_gold_actions(maze)

In [ ]:
get_supervised_states(maze)

In [ ]:
function get_all_data_up_to(N::Int)
    """
    Generates a list of tuples, where each tuple contains supervised states
    and gold actions for a specific configuration. There are 2N tuples in the
    list.
    """
    data = []
    for i=1:N
        for goal in [0, 1]
            maze = TMaze(i,goal,(0,0));
            states = get_supervised_states(maze)
            actions = get_gold_actions(maze)
            push!(data, (states, actions))
        end
    end
    return data
end

In [ ]:
data = get_all_data_up_to(3)

## Problem 1. Implement the LSTM Agent
You need to implement initweights function which takes hidden, world size and number of actions, and returns the whole model as `Array{Any}` julia data type.

In [ ]:
# model definition
struct Linear
    w
    b
end

# Initializer for a softmax classifier
function Linear(num_inputs::Int, num_outputs::Int; init=gaussian, atype=Knet.atype())
    # Your code here
end

function (l::Linear)(x)
    # Your code here
end

struct LSTMNet
    w
    b
end

# Hint you mave take a look main function below to better understand its calling convention
# Remember that, forget gate bias should be ones instead of zeros
function LSTMNet(num_inputs::Int, hidden_size::Int; init=gaussian, atype=Knet.atype())
    # Your code here
end

# lstm procedure is given to you
# weight * input .+ bias, concatenated weights for computational efficiency
# You should use this function in your LSTM module call
function lstm(weight, bias, hidden, cell, input)
    gates   = weight * vcat(hidden,input) .+ bias
    hsize   = size(hidden,1)
    forget  = sigm.(gates[1:hsize,:])
    ingate  = sigm.(gates[1+hsize:2hsize,:])
    outgate = sigm.(gates[1+2hsize:3hsize,:])
    change  = tanh.(gates[1+3hsize:end,:])
    cell    = cell .* forget + ingate .* change
    hidden  = outgate .* tanh.(cell)
    return (hidden, cell)
end

function (l::LSTMNet)(x, prev_hidden, prev_cell)
    # Your code here
end

mutable struct LSTMAgent
    lstm::LSTMNet # a lstm network
    linear::Linear # a linear layer on top of the lstm network
    state # Array{Any}(undef, 2) for hidden and cell states
    atype # Array{Float32} or KnetArray{Float32}. Defaults to Knet.atype()
end

function LSTMAgent(hidden_size::Int, world_dim::Int=3, num_actions::Int=4; atype=Knet.atype())
    # Your code here
end

# resets the hidden and cell states
function reset!(model::LSTMAgent)
    # Your code here
end

# before calling the model you should reset the hidden and cell states
# model gets world state(s) and applies the lstm function to each state in the world_states
# and predicts a score vector for actions.
# Don't forget to convert the data to atype!
function (model::LSTMAgent)(world_states)
    scores = []
    # Your code here
    return scores
end

## Problem 2. Implement Loss function
That function basically takes the predictions and returns the negative log-likelihood of these predictions as loss.
Hint: You may have a look Knet's at ```nll``` function

In [ ]:
# our loss function
function loss(model, world_states, gold_actions)
    total = 0.0
    scores = model(world_states)

    # Your code here
    lossval = total/length(gold_actions)
    return lossval
end

In [ ]:
function train!(model, world_states, gold_actions)
    L = @diff loss(model, world_states, gold_actions)
    for p in params(model)
        g = grad(L, p)
        update!(value(p), g, p.opt)
    end
    return value(L)
end

## Problem 3. Implement accuracy function
This function takes a model and a data array, and returns an array of 1's and 0's for the correctness of that action set.
In order for the action set's result to be 1, each move has to be correct (same with the gold action in that state).
Hint: You will need ```argmax``` function
Don't forget to convert the model output to Array, since it can be a KnetArray if you are using GPU.

In [ ]:
# possible helpful procedures: vec
function path_accuracy(model, data)
    accuracies = []
    for (world_states, gold_actions) in data
        ncorrect = 0
        reset!(model)
        scores = model(world_states)
        # ncorrect must be equal to the path length (i.e. length of the gold actions)

        # Your code here

        push!(accuracies, ncorrect == length(gold_actions) ? 1.0 : 0.0)
    end

    return accuracies
end

In [ ]:
function train_until_all_success(model, data; maxe=50000)
    """
    Gets model parameters w, initial states s, optimizers opt and data.
    It trains the model untill the accuracy on the data reach to 1.0.
    """
    experiment = []
    for i=1:maxe
        world_states, gold_actions = rand(data) #sample data
        reset!(model)
        train!(model, world_states, gold_actions)

        accuracies = path_accuracy(model, data)
        total = sum(accuracies)/length(accuracies)
        push!(experiment, total)
        print("\r Number of Instances: $i Acc: $(@sprintf("%.3f", total))")
        if total == 1.0
            break
        end
    end
    return experiment
end

In [ ]:
data = get_all_data_up_to(10);

In [ ]:
# Model with 4 hidden_size
Knet.seed!(1)
HIDDEN = 4

model_4 = LSTMAgent(HIDDEN, atype=Array{Float32})
for p in params(model_4)
    p.opt = Adam()
end

Random.seed!(1)
hidden_4 = train_until_all_success(model_4, data; maxe=100000);

In [ ]:
# Model with 8 hidden_size
Knet.seed!(1)
HIDDEN = 8

model_8 = LSTMAgent(HIDDEN, atype=Array{Float32})
for p in params(model_8)
    p.opt = Adam()
end

Random.seed!(1)
hidden_8 = train_until_all_success(model_8, data; maxe=100000);

In [ ]:
# Model with 16 hidden_size
Knet.seed!(1)
HIDDEN = 16

model_16 = LSTMAgent(HIDDEN, atype=Array{Float32})
for p in params(model_16)
    p.opt = Adam()
end

Random.seed!(1)
hidden_16 = train_until_all_success(model_16, data; maxe=100000);

In [ ]:
plot([1:length(hidden_4), 1:length(hidden_8), 1:length(hidden_16)],
    [hidden_4, hidden_8, hidden_16],
    label=["hidden_4" "hidden_8" "hidden_16"],
    xlabel="Number of instances", ylabel="Accuracy", title="Hidden size comparison")

In [ ]:
limit = 50
test_data = get_all_data_up_to(limit);

accuracies = path_accuracy(model_4, test_data)
N_4 = map(x->sum(accuracies[(x-1)*2+1:x*2])/2, 1:limit);

accuracies = path_accuracy(model_8, test_data)
N_8 = map(x->sum(accuracies[(x-1)*2+1:x*2])/2, 1:limit);

accuracies = path_accuracy(model_16, test_data)
N_16 = map(x->sum(accuracies[(x-1)*2+1:x*2])/2, 1:limit);

plot([N_4 N_8 N_16], linetype=:scatter, m=[:circle :cross :star5],
    label=["hidden_4" "hidden_8" "hidden_16"],
    xlabel="Corridor Length", ylabel="Accuracy", title="Generalization")

hidden_4 and hidden_8 models can only handle up to corridor length equals to 10. However, hidden_16 is able to solve the task. Let's inspect the behaviour of this model.

## Problem 4. Implement Play function
`play` function gets a maze, trained model parameters w and initial states s. It takes action using the model until either the agent reaches the final state or exceeds the maximum action limit. The function returns the actions taken, hidden and cell states of the lstm.
Hint: part of the code is the same as in path_accuracy function.
Don't forget to convert the model prediction to Array, since it can be a KnetArray if you are using GPU.

In [ ]:
function play(maze, model; max_actions=20)
    res = 0
    action_count = 0
    hiddens = []
    cells = []
    actions = []
    draw(maze)
    reset!(model)
    while !(res != 0 || action_count >= max_actions)
        sleep(1) # sleep 1 second
        x = get_state(maze) # get state
        # Your code here

        res = step(maze, action) # prediction from the model
        draw(maze)
        action_count += 1
    end

    return actions, hiddens, cells
end

In [ ]:
maze = TMaze(5, 0, (0, 0))
actions_0, hiddens_0, cells_0 = play(maze, model_16);

### Hidden and cell state visualizations

In [ ]:
xs = [string("state", i) for i = 1:maze.length+1]
ys = [string("unit", i) for i = 1:16]
heatmap(xs, ys, hcat(hiddens_0...), title="Hidden states for a maze(length=5) and the goal is north")

In [ ]:
xs = [string("state", i) for i = 1:maze.length+1]
ys = [string("unit", i) for i = 1:16]
heatmap(xs, ys, hcat(cells_0...), title="Cell states for a maze(length=5) and the goal is north")

In [ ]:
maze = TMaze(5, 1, (0, 0))
actions_1, hiddens_1, cells_1 = play(maze, model_16);

In [ ]:
xs = [string("state", i) for i = 1:maze.length+1]
ys = [string("unit", i) for i = 1:16]
heatmap(xs, ys, hcat(hiddens_1...), title="Hidden states for a maze(length=5) and the goal is south")

In [ ]:
xs = [string("state", i) for i = 1:maze.length+1]
ys = [string("unit", i) for i = 1:16]
heatmap(xs, ys, hcat(cells_1...), title="Cell states for a maze(length=5) and the goal is south")

Until the t-junction, the model preserves the hidden state and at the t-junction it switches hidden state for the final prediction.

### Some weight visualization

In [ ]:
xs = [string("x", i) for i = 1:3]
ys = [string("w", i) for i = 1:16]
heatmap(xs, ys, value(model_16.lstm.w)[1:16,17:19], title="Forget gate weight used in W_forget * x_t")

In [ ]:
xs = [string("x", i) for i = 1:3]
ys = [string("w", i) for i = 1:16]
heatmap(xs, ys, value(model_16.lstm.w)[17:32,17:19], title="Input gate weight used in W_input * x_t")

In [ ]:
xs = [string("h", i) for i = 1:16]
ys = [string("w", i) for i = 1:16]
heatmap(xs, ys, value(model_16.lstm.w)[17:32,1:16], title="Input gate weight used in W_input * prev_hidden")

Do not bother making any comment about this last heatmap.

Now discuss the findings from the figures.

Your Comment:

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*